# **Thư viện**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error



RecursionError: maximum recursion depth exceeded

# **Dữ liệu**

In [ ]:
df = pd.read_csv(r"D:\eur-vnd-exchange-rate\data\processed\EUR_VND_Exchange_Rate.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

y = df['Sell']
y.plot(title="Tỷ giá EUR/VND")
plt.show()

In [ ]:
# Phân tích thành phần của chuỗi thời gian
decomposition = seasonal_decompose(df['Sell'], model='additive', period=120)
plt.figure(figsize=(14, 10))
decomposition.plot()
plt.tight_layout()
plt.show()

# **Kiểm tra tính dừng (ADF Test)**

In [ ]:
result = adfuller(y)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

if result[1] < 0.05:
    print("Chuỗi dừng")
else:
    print("Chuỗi không dừng, cần lấy sai phân")


## **Lấy sai phân**

In [ ]:
y_diff = y.diff().dropna()
y_diff.plot(title="Chuỗi sau khi sai phân")
plt.show()

In [ ]:
result = adfuller(y_diff)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

if result[1] < 0.05:
    print("Chuỗi dừng")
else:
    print("Chuỗi không dừng, cần lấy sai phân")

=> d = 1

# **ACF và PACF để chọn p, q**

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

_ = plot_acf(y_diff, ax=ax1, lags=30, zero=False)
_ = plot_pacf(y_diff, ax=ax2, lags=30, zero=False)

ax1.set_ylim(-0.1, 0.1)
ax2.set_ylim(-0.1, 0.1)

plt.tight_layout()
plt.show()


ACF loại bỏ lag0 tất cả đều trong vùng xanh 
=> q = 0  

Tương tự, các giá trị PACF đều nằm trong vùng xanh, tức là không có độ trễ nào nổi bật rõ ràng
=> p = 0

(p, d, q) = (0, 1, 0)

# **ARIMA**

In [ ]:
train_size = int(len(y) * 0.8)
train, test = y[:train_size], y[train_size:]

train.plot(title="Dữ liệu huấn luyện")
test.plot(title="Dữ liệu kiểm tra")
plt.show()

In [ ]:
# Fit mô hình ARIMA(0, 1, 0) trên tập huấn luyện
model = ARIMA(train, order=(1, 1, 6))
result = model.fit()

print(result.summary())


# **Đánh giá mô hình**

In [ ]:
# Dự báo trên tập kiểm tra (test)
y_pred = result.forecast(steps=len(test))

# Đánh giá mô hình
MAE = mean_absolute_error(test, y_pred)
MSE = mean_squared_error(test, y_pred)
RMSE = np.sqrt(MSE)
MAPE = np.mean(np.abs((test - y_pred) / test)) * 100

print(f"MAE: {MAE}")
print(f"MSE: {MSE}")
print(f"RMSE: {RMSE}")
print(f"MAPE: {MAPE}%")

# Vẽ biểu đồ so sánh thực tế và dự báo
plt.figure(figsize=(10, 6))
plt.plot(test, label='Thực tế', color='blue')
plt.plot(y_pred, label='Dự đoán', color='red')
plt.title('So sánh dự đoán và thực tế trên tập kiểm tra')
plt.xlabel('Thời gian')
plt.ylabel('Giá trị')
plt.legend()
plt.show()
